In [1]:
# Library imports
import os
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import nltk
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.preprocessing.text import Tokenizer
from deep_translator import MyMemoryTranslator
import numpy as np
nltk.download('stopwords')
nltk.download('wordnet')
print(os.getcwd())

c:\Users\User\Desktop\ER_NUI\SentimentAnalysis


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#https://github.com/PoorvaRane/Emotion-Detector/blob/master/ISEAR.csv
#https://github.com/sinmaniphel/py_isear_dataset/blob/master/isear.csv

In [ ]:
def transform_data():
    with open("data.csv", "r", ) as input_file, open("transformed_data.csv", "w") as output_file:
        for line in input_file:
            cleaned = line.strip()
            pos = cleaned.find(',')
            emotion = cleaned[:pos]
            # replace the first , after emotion
            if emotion in ["joy","fear","anger","sadness","disgust","shame","guilt"]:
                cleaned = '%s%s%s'%(cleaned[:pos],";",cleaned[pos+1:])
            print(cleaned,len(cleaned))
            if cleaned[len(cleaned)-1] == ",":
                cleaned = '%s%s%s'%(cleaned[:len(cleaned)-1],";",cleaned[len(cleaned):])
            
            output_file.write(f'{cleaned}\n')    

In [ ]:
with open('transformed_data.csv', 'r') as f, open("transformed_data_text.csv", "w") as output_file:
    reader = csv.reader(f, delimiter=';')
    for row in reader:
        #print(row)
        line=row[1].replace("\n"," ")
        emo = row[0]
        output_file.write(f'{emo};{line}\n')

In [ ]:
from_language, to_language = 'en', 'pt-PT'
with  open("ISEAR_2_formatted.csv", "r") as file, open("ISEAR_portuguese_2.csv", "a", encoding="utf-8") as output_file:
    reader = csv.reader(file, delimiter=';')
    reader2 = csv.reader(output_file, delimiter=';')
    size = 0
    for idx, row in tqdm(enumerate(reader)):
        if idx < size:
            continue
        else:
            input_text = row[1]
            #translated_text = tss.google(input_text, reset_host_url=None, from_language=from_language, to_language=to_language)
            #translated_text = input_text.translate(to='pt-PT', from_lang='en')
            translated_text = MyMemoryTranslator(source='english', target='portuguese').translate(input_text) 

            output_file.write(f'{row[0]};{translated_text}\n')

In [ ]:
# Load the dataset
dataset = pd.read_csv("ISEAR_portuguese.csv",sep=";")
texts = dataset["text"].values
classes = dataset["sentiment"].values
# Load stop words
stop_words = set(stopwords.words('portuguese'))
tokenizer = Tokenizer()
X_train, X_test, y_train, y_test = train_test_split(texts, classes, test_size=0.03, random_state=2)
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1
seq_lengths = [len(seq.split()) for seq in X_train]
max_len = int(np.percentile(seq_lengths, 95))
print(vocab_size, max_len)

In [ ]:
# MultinomialNB

vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                             max_features = 10000, stop_words=stop_words)

train_data_features = vectorizer.fit_transform(X_train)
print("########## Dictionary built ##########\n")
test_data_features = vectorizer.transform(X_test)

model = MultinomialNB()
model.fit(train_data_features, y_train)
MNBpredict = model.predict(test_data_features)
print("Accuracy: {0:.2f}%".format(metrics.accuracy_score(y_test, MNBpredict)*100))

In [ ]:
# Evaluate
X_test = vectorizer.transform(["Estou triste"])
y_pred = model.predict_proba(X_test)
print(y_pred, model.classes_)

In [ ]:
# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_len))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', accuracy)


In [16]:
dataset = pd.read_csv("ISEAR_2_formatted.csv",sep=";")
sentiments = dataset["sentiment"].values
with open("ISEAR_only_text_eng.csv", "r", encoding="utf-8") as file, open("ISEAR_only_text_eng_final.csv", "w", encoding="utf-8") as output_file:
    reader = csv.reader(file, delimiter='\n')
    for idx,row in enumerate(reader):
        output_file.write(f'{sentiments[idx]};{row[0]}\n')